In [172]:
%load_ext sql
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [173]:
%sql ROLLBACK

Running query in 'duckdb:///:memory:'

Success


In [174]:
%%sql


CREATE OR REPLACE TABLE circuits AS SELECT * FROM 'seeds/circuits.csv';
CREATE OR REPLACE TABLE constructors AS SELECT * FROM 'seeds/constructors.csv';
CREATE OR REPLACE TABLE drivers AS SELECT * FROM 'seeds/drivers.csv';
CREATE OR REPLACE TABLE races AS SELECT * FROM 'seeds/races.csv';
CREATE OR REPLACE TABLE results AS SELECT * FROM 'seeds/results.csv';
CREATE OR REPLACE TABLE status AS SELECT * FROM 'seeds/status.csv';

CREATE OR REPLACE TABLE qualifying AS SELECT * FROM 'seeds/qualifying.csv';
CREATE OR REPLACE TABLE pit_stops AS SELECT * FROM 'seeds/pit_stops.csv';
CREATE OR REPLACE TABLE lap_times AS SELECT * FROM 'seeds/lap_times.csv';

Running query in 'duckdb:///:memory:'

Count


# Project "Finding driver talents in F1".

They say that F1 bolids are 80% of success and driver is only responsible for 20%. In this research I will try to analize seeds from F1 2000-2023 and try to find those drivers, who were hidden talents. That means I will try to subside the impact of 'good car' and 'standarize' drivers by their achievements. To do that, I will first find drivers who were most dominatin in each team. Because 2 drivers in each teams each year have the same car, then driver who is dominating second driver by most points will be considered as driver talent despite car. Second calculation which will allow me to know the metric will be finding the drivers who gained the most places in sunday's race. This shows which driver despite his car, had  the biggest talent to drive

## Combining tables into one big table

First step of analysis was to create one table containing combined data from tables from data. It will be easier later to not joining all tables again and again but only once here. We also filtered out races before year 2000, as we will focus on new F1. 

In [175]:
%%sql
CREATE OR REPLACE TABLE team_stats AS
WITH main_table as(
SELECT r.driverId,r.raceId,r.constructorId,r.points as points,r.position,CONCAT(d.forename,' ',d.surname) as driver_name,c.name as team,ra.name as race_name,ra.year as race_year,ra.date as race_date,SUM(r.points) OVER (PARTITION BY r.raceId,r.constructorId) as team_points
from results as r
inner join constructors as c on r.constructorId=c.constructorId
inner join drivers as d on r.driverId=d.driverId
inner join races as ra on r.raceId=ra.raceId
where race_year>=2000)

SELECT * FROM main_table

Running query in 'duckdb:///:memory:'

Count


## teamwars / Window Functions

Pierwsze zbadamy czy obydwaj kierowcy w zespole rowno ciagneli swoj zespol. Jesli kierowca 1 zdobyl 25 punktow a kierowca 2, to kier.1 ma 100% a 2 0%. Jesli kier1.=25pkt a kier2.=18pkt, wtedy kier1=58%

In [176]:
%%sql


WITH team_percentage as (
    SELECT 
    *,
    CASE
        WHEN team_points=0 THEN 0
        ELSE points/team_points 
    END as percentage_team_points
    from team_stats
) 

SELECT * FROM team_percentage

Running query in 'duckdb:///:memory:'

driverId,raceId,constructorId,points,position,driver_name,team,race_name,race_year,race_date,team_points,percentage_team_points
5,1,1,0.0,\N,Heikki Kovalainen,McLaren,Australian Grand Prix,2009,2009-03-29,0.0,0.0
1,1,1,0.0,\N,Lewis Hamilton,McLaren,Australian Grand Prix,2009,2009-03-29,0.0,0.0
12,2,4,0.0,13,Nelson Piquet Jr.,Renault,Malaysian Grand Prix,2009,2009-04-05,0.0,0.0
4,2,4,0.0,11,Fernando Alonso,Renault,Malaysian Grand Prix,2009,2009-04-05,0.0,0.0
22,2,23,2.0,5,Rubens Barrichello,Brawn,Malaysian Grand Prix,2009,2009-04-05,7.0,0.2857142857142857
18,2,23,5.0,1,Jenson Button,Brawn,Malaysian Grand Prix,2009,2009-04-05,7.0,0.7142857142857143
67,4,5,0.0,17,Sébastien Buemi,Toro Rosso,Bahrain Grand Prix,2009,2009-04-26,0.0,0.0
7,4,5,0.0,13,Sébastien Bourdais,Toro Rosso,Bahrain Grand Prix,2009,2009-04-26,0.0,0.0
17,4,9,0.0,11,Mark Webber,Red Bull,Bahrain Grand Prix,2009,2009-04-26,8.0,0.0
20,4,9,8.0,2,Sebastian Vettel,Red Bull,Bahrain Grand Prix,2009,2009-04-26,8.0,1.0


Teraz obliczumy który kierowca miał najwyzszą średnią udziału punktów w swoim zespole w kazdym roku.

In [177]:
%%sql
CREATE OR REPLACE TABLE driver_dominance AS
WITH team_percentage AS (
    SELECT 
        *,
        -- Zabezpieczenie przed dzieleniem przez zero
        CASE
            WHEN team_points = 0 THEN 0
            ELSE points / team_points 
        END as percentage_team_points
    FROM team_stats
)

SELECT
    driver_name,
    race_year,
    team,
    COUNT(raceId) as race_count,
    SUM(points) as points_sum,
    -- To będzie nasza kluczowa kolumna do rankingu:
    AVG(percentage_team_points) as avg_dominance 
FROM team_percentage
GROUP BY driver_name, race_year, team
HAVING race_count > 5 -- Aby usunąć wpływ kierowców rezerwowych
ORDER BY avg_dominance DESC;

Running query in 'duckdb:///:memory:'

Count


Max Verstappen in season 2024 was the most dominującym kierowcą w swoim zespole. Co ciekawe Robert Kubica w sezonie 2010 miał 3 wynik w tym zestawieniu. 

## Ilość zdobytych miejsc  w niedzielnym wyścigu

What is more we need to subside races where car was broken. It is not the driver fault. But when the loss of positions was because of driver (etc. crash) then it should be counted 

In [178]:
%%sql
select * from status
where statusId in [1,2,3,4,11,12,13,14,15,16,17,18,19,85]
order by statusId 

Running query in 'duckdb:///:memory:'

statusId,status
1,Finished
2,Disqualified
3,Accident
4,Collision
11,+1 Lap
12,+2 Laps
13,+3 Laps
14,+4 Laps
15,+5 Laps
16,+6 Laps


In [185]:
%%sql
CREATE OR REPLACE TABLE racecraft_stats AS
WITH position_calc AS(
SELECT
    t.driver_name,
    t.driverId,
    t.race_year,
    t.team,
    r.grid,
    r.positionOrder,
    r.statusId,
    (r.grid-r.positionOrder) as position_gained
from team_stats as t
inner join results as r on t.raceId=r.raceId AND t.driverId=r.driverId
where r.statusId in [1,2,3,4,11,12,13,14,15,16,17,18,19,85])

select driverId,driver_name,race_year,team,ROUND(AVG(position_gained),2) as avg_position_gained,count(*) as races_count from position_calc
GROUP BY driverId,driver_name,race_year,team
HAVING races_count > 15
ORDER BY avg_position_gained DESC

Running query in 'duckdb:///:memory:'

Count


In [186]:
%%sql
select * from racecraft_stats

Running query in 'duckdb:///:memory:'

driverId,driver_name,race_year,team,avg_position_gained,races_count
33,Tiago Monteiro,2005,Jordan,4.89,18
818,Jean-Éric Vergne,2012,Toro Rosso,3.89,18
838,Stoffel Vandoorne,2018,McLaren,3.89,18
816,Jérôme d'Ambrosio,2011,Virgin,3.71,17
21,Giancarlo Fisichella,2004,Sauber,3.56,18
825,Kevin Magnussen,2016,Renault,3.41,17
840,Lance Stroll,2019,Racing Point,3.32,19
8,Kimi Räikkönen,2005,McLaren,3.31,16
831,Felipe Nasr,2016,Sauber,3.29,17
1,Lewis Hamilton,2014,Mercedes,3.25,16


Grupowanie jaki kierowca w jakim sezonie miał najlepszą średnią zdobytych miejsc

Dodaktowo przeprowadzimy normalizacje wyniku avg_position_gained aby moc potem obliczyc poprawnie wskaznik. Zrobię to wedlug wzoru (x -min(x))/(max(x)-min(x))

In [194]:
%%sql
WITH stats_per_year AS (
    -- 1. Łączymy dane i liczymy zakresy (Min/Max) dla każdego roku
    -- Dzięki temu porównujemy kierowców tylko z ich rywalami z danego sezonu
    SELECT 
        d.driver_name,
        d.race_year,
        d.team,
        d.avg_dominance, -- To jest ułamek np. 0.8 (80%)
        r.avg_position_gained,      -- To jest liczba np. 3.2
        
        -- Magia okien: Szukamy skrajnych wartości w danym roku
        MIN(r.avg_position_gained) OVER (PARTITION BY d.race_year) as min_gain,
        MAX(r.avg_position_gained) OVER (PARTITION BY d.race_year) as max_gain
        
    FROM driver_dominance as d
    INNER JOIN racecraft_stats as r 
        ON d.driver_name = r.driver_name 
        AND d.race_year = r.race_year
        AND d.team = r.team
),

normalized_scores AS (
    SELECT 
        *,
        -- 2. Zamieniamy Dominację na punkty 0-100
        -- (Mnożymy razy 100, ale ograniczamy do 100, żeby nikt nie miał 120%)
        LEAST(avg_dominance * 100, 100) as score_dominance,
        
        -- 3. Zamieniamy Wyprzedzanie na punkty 0-100 (Wzór Min-Max)
        -- Zabezpieczenie NULLIF na wypadek gdyby max = min (jeden kierowca w roku)
        ((avg_position_gained - min_gain) / NULLIF(max_gain - min_gain, 0)) * 100 as score_racecraft
        
    FROM stats_per_year
)

SELECT 
    race_year,
    driver_name,
    team,
    ROUND(score_dominance, 1) as points_dominance_in_team,
    ROUND(score_racecraft, 1) as points_overtake,
    
    -- 4. FINAŁ: Średnia ważona (70% Speed + 30% Overtake)
    ROUND(
        (score_dominance * 0.7) + (COALESCE(score_racecraft, 0) * 0.3)
    , 2) as TOTAL_RATING
    
FROM normalized_scores
WHERE score_racecraft IS NOT NULL -- Odsiewamy błędy dzielenia
ORDER BY TOTAL_RATING DESC
LIMIT 20;

Running query in 'duckdb:///:memory:'

race_year,driver_name,team,points_dominance_in_team,points_overtake,TOTAL_RATING
2024,Max Verstappen,Red Bull,77.2,72.4,75.73
2012,Kimi Räikkönen,Lotus F1,74.5,77.5,75.4
2023,Max Verstappen,Red Bull,69.8,83.1,73.77
2013,Kimi Räikkönen,Lotus F1,60.6,100.0,72.43
2010,Robert Kubica,Renault,73.3,69.4,72.1
2013,Fernando Alonso,Ferrari,70.3,68.8,69.86
2006,Michael Schumacher,Ferrari,58.0,96.7,69.65
2012,Fernando Alonso,Ferrari,70.9,64.4,68.97
2014,Fernando Alonso,Ferrari,70.0,60.9,67.3
2023,Fernando Alonso,Aston Martin,68.9,63.4,67.26


## Obliczenie wskaźnika best driver

Aby znaleźć najlepszego kierowcę wykorzystam powyzsze dwa wyniki wedlug formuly: (dominance * 0,7)+(normalized_position_gain*0,3)

In [ ]:
%%sql
WITH stats_per_year AS (
    SELECT 
        d.driver_name,
        d.race_year,
        d.team,
        d.avg_dominance, 
        r.avg_position_gained,     
        
        MIN(r.avg_position_gained) OVER (PARTITION BY d.race_year) as min_gain,
        MAX(r.avg_position_gained) OVER (PARTITION BY d.race_year) as max_gain
        
    FROM driver_dominance as d
    INNER JOIN racecraft_stats as r 
        ON d.driver_name = r.driver_name 
        AND d.race_year = r.race_year
        AND d.team = r.team
),

normalized_scores AS (
    SELECT 
        *,
        LEAST(avg_dominance * 100, 100) as score_dominance,
        ((avg_position_gained - min_gain) / NULLIF(max_gain - min_gain, 0)) * 100 as score_racecraft
        
    FROM stats_per_year
)

SELECT 
    race_year,
    driver_name,
    team,
    ROUND(score_dominance, 1) as points_dominance_in_team,
    ROUND(score_racecraft, 1) as points_overtake,
    
    ROUND(
        (score_dominance * 0.7) + (COALESCE(score_racecraft, 0) * 0.3)
    , 2) as TOTAL_RATING
    
FROM normalized_scores
WHERE score_racecraft IS NOT NULL 
ORDER BY TOTAL_RATING DESC
LIMIT 20;

Running query in 'duckdb:///:memory:'

race_year,driver_name,team,points_dominance_in_team,points_overtake,TOTAL_RATING
2024,Max Verstappen,Red Bull,77.2,72.4,75.73
2012,Kimi Räikkönen,Lotus F1,74.5,77.5,75.4
2023,Max Verstappen,Red Bull,69.8,83.1,73.77
2013,Kimi Räikkönen,Lotus F1,60.6,100.0,72.43
2010,Robert Kubica,Renault,73.3,69.4,72.1
2013,Fernando Alonso,Ferrari,70.3,68.8,69.86
2006,Michael Schumacher,Ferrari,58.0,96.7,69.65
2012,Fernando Alonso,Ferrari,70.9,64.4,68.97
2014,Fernando Alonso,Ferrari,70.0,60.9,67.3
2023,Fernando Alonso,Aston Martin,68.9,63.4,67.26
